<a href="https://colab.research.google.com/github/DiAnCaceres/Minor-Machine-Learning/blob/main/Proyecto_ML/Entrega_02_Scrub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Proyecto Final - Reconocimiento de Actividad Humana (HAR)


In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

###Entrega 01 - OBTAIN

Incluir todo el código de la Etapa 01 - OBTAIN creada en la semana anterior

In [2]:
url = "https://drive.google.com/file/d/1Nacn6CvB6xcltzx-_hp5AHw2YkME1iiB/view?usp=share_link" #link del dataset(link hacia .txt que esta in mi gdrive)
url = "https://drive.google.com/uc?id=" + url.split('/')[-2]
dataset = pd.read_csv(url, on_bad_lines='skip', header = None) #se agrego on_bad_lines skip ya que daba error al cargar el dataset
dataset

,0,1,2,3,4,5
0,33,Jogging,49105962326000,-0.694638,12.680544,0.50395286;
1,33,Jogging,49106062271000,5.012288,11.264028,0.95342433;
2,33,Jogging,49106112167000,4.903325,10.882658,-0.08172209;
3,33,Jogging,49106222305000,-0.612916,18.496431,3.0237172;
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164;
...,...,...,...,...,...,...
1086461,19,Sitting,131623331483000,9.000000,-1.570000,1.69;
1086462,19,Sitting,131623371431000,9.040000,-1.460000,1.73;
1086463,19,Sitting,131623411592000,9.080000,-1.380000,1.69;
1086464,19,Sitting,131623491487000,9.000000,-1.460000,1.73;


###Entrega 02 - SCRUB

Eliminar filas con datos faltantes

*Uno de los pasos iniciales en la limpieza de datos consiste en eliminar aquellas filas que contengan datos faltantes. Estas líneas generalmente no aportan información y pueden complicar la etapa de entrenamiento y validación de muchos modelos predictivos.*


In [3]:
dataset.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    1
dtype: int64

In [5]:
dataset = dataset.dropna(how='any',axis=0) 
dataset.isnull().sum()

user        0
activity    0
time        0
x           0
y           0
z           0
dtype: int64

Agregar al DataFrame los nombres de las columnas 

*En ocasiones al crear los data frames el sistema asigna nombres genéricos y correlativos a las columnas. Es recomendable renombrar las columnas por nombres significativos para favorecer el análisis posterior de los datos.*

In [4]:
dataset.columns = ["user", "activity", "time", "x", "y", "z"]


de la manera que se importo el dataset, quedaron punto y comas en la columna z por lo que se deben quitar

In [6]:
dataset["z"] = dataset["z"].str.replace(";", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Visualizar las 5 primeras filas del DataFrame (.head())

*Siempre es buena medida visualizar algunas líneas del dataframe para tener una idea concreta del contenido.*


In [7]:
dataset.head()

,user,activity,time,x,y,z
0,33,Jogging,49105962326000,-0.694638,12.680544,0.50395286
1,33,Jogging,49106062271000,5.012288,11.264028,0.95342433
2,33,Jogging,49106112167000,4.903325,10.882658,-0.08172209
3,33,Jogging,49106222305000,-0.612916,18.496431,3.0237172
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164


Buscar datos nulos y eliminarlos si es que existen

In [ ]:
#se realizo al principio de la entrega 2

user         0
activity     0
timestamp    0
x-accel      0
y-accel      0
z-accel      1
dtype: int64

Visualizar los tipo de datos a los que pertenece cada columna (.info())

*Es necesario tener un detalle de cuantas variables están considerando y el tipo de datos que pertenecen para verificar si es posible utilizarla directamente para el modelo predictivo o si es necesario realizar una transformación del tipo de dato.*


In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1086465 entries, 0 to 1086465
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   user      1086465 non-null  int64  
 1   activity  1086465 non-null  object 
 2   time      1086465 non-null  int64  
 3   x         1086465 non-null  float64
 4   y         1086465 non-null  float64
 5   z         1086465 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 58.0+ MB


Cambiar el tipo de datos de las variables predictoras a valores reales, para realizar la clasificación 

*Muchas de las variables contenidas en el DataFrame originalmente se encuentran en el tipo de dato 'object'. En ese formato las variables no pueden ser interpretadas como vectores. Recordemos que la mayoría de los modelos de Machine Learning estudiados operan en el espacio vectorial. Por esta razón las variables que formarán parte del modelo predictivo deben ser transformadas a un formato numérico como 'Float' o 'integer'.*

Para este proyecto en particular se pide transformar  al tipo de dato Float, las variables del acelerómetro en el eje $x$, $y$ y $z$



In [9]:
 dataset['z'] = dataset["z"].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Imprimir los tipos de datos de las variables del DataFrame

*Se recomienda volver a ejecutar el comando info para verificar que efectivamente las variables cambiaron su tipo de dato y que ahora se pueden procesar con los modelos predictivos.*


In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1086465 entries, 0 to 1086465
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   user      1086465 non-null  int64  
 1   activity  1086465 non-null  object 
 2   time      1086465 non-null  int64  
 3   x         1086465 non-null  float64
 4   y         1086465 non-null  float64
 5   z         1086465 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 58.0+ MB


Eliminar las columnas que no se utilizaran para la clasificación

*En este caso deberíamos conservar la actividad respectiva y las variables asociadas al acelerómetro*


In [12]:
df = dataset.drop(["user" ,"time"],axis=1)
df

,activity,x,y,z
0,Jogging,-0.694638,12.680544,0.503953
1,Jogging,5.012288,11.264028,0.953424
2,Jogging,4.903325,10.882658,-0.081722
3,Jogging,-0.612916,18.496431,3.023717
4,Jogging,-1.184970,12.108489,7.205164
...,...,...,...,...
1086461,Sitting,9.000000,-1.570000,1.690000
1086462,Sitting,9.040000,-1.460000,1.730000
1086463,Sitting,9.080000,-1.380000,1.690000
1086464,Sitting,9.000000,-1.460000,1.730000


Utilizar el comando *value_counts* para obtener un detalle de cuantas instancias se tienen por actividad. El objetivo consiste en identificar la presencia del balance o desbalance en las categorías a modelar


In [ ]:
df.value_counts("activity")

activity
Walking       424397
Jogging       342176
Upstairs      122869
Downstairs    100427
Sitting        59939
Standing       48395
dtype: int64

Este proyecto considera datos con un desbalance de clases. Es necesario balancearlos para que los modelos operen de manera adecuada. Existen muchas maneras de realizar un balance, pero en este proyecto utilizaremos una estrategia sencilla e intuitiva. Simplemente debe identificar la actividad que contenga el número menor de instancias. Se debe realizar un muestreo para las otras actividades utilizando ese valor. De esta manera todas las clases tendrán la misma cantidad de instancias. 



In [34]:
g = df.groupby('activity') #se separan en grupos de actividades
min = g.size().min() #se obtiene el valor minimo del tamaño de los grupos
Walking = g.get_group("Walking").head(min) #se obtiene el grupo especifico y solo se ocupan los primeros n datos, donde n es la minima catidad de datos
Jogging = g.get_group("Jogging").head(min)
Upstairs = g.get_group("Upstairs").head(min)
Downstairs = g.get_group("Downstairs").head(min)
Sitting = g.get_group("Sitting").head(min)
Standing = g.get_group("Standing").head(min)
df_ds= pd.concat([Walking, Jogging, Upstairs, Downstairs, Sitting, Standing]) # se concatenan todas los grupos para volver a formar el dataframe ya con downsampling

Establecer el número de instancias y dimensiones del data frame resultante *(shape)*


In [40]:
df_ds.shape

(290364, 4)

Volver a imprimir la cantidad de instancias por categoría para verificar que todo esté en orden. *(value_counts)*

In [42]:
df_ds.value_counts("activity") #se comprueba si se tuvo exito

activity
Downstairs    48394
Jogging       48394
Sitting       48394
Standing      48394
Upstairs      48394
Walking       48394
dtype: int64

Volver a visualizar las cinco primeras líneas del DataFrame para revisar que todo esté en orden *(head)*


In [41]:
df_ds.head(5)

,activity,x,y,z
597,Walking,0.844462,8.008764,2.792171
598,Walking,1.116869,8.621680,3.786457
599,Walking,-0.503953,16.657684,1.307553
600,Walking,4.794363,10.760075,-1.184970
601,Walking,-0.040861,9.234595,-0.694638


Separar las variables predictoras (X) de las ctegorias o etiquetas (y)

In [62]:
X = df_ds.iloc[:,1:] #se toman todos los datos a la derecha de la primera columna
y = df_ds.iloc[:,:1] #se toma solo la primera columna

In [64]:
X

,x,y,z
597,0.844462,8.008764,2.792171
598,1.116869,8.621680,3.786457
599,-0.503953,16.657684,1.307553
600,4.794363,10.760075,-1.184970
601,-0.040861,9.234595,-0.694638
...,...,...,...
1083927,-4.990000,8.390000,-0.380000
1083928,-5.130000,8.240000,-0.420000
1083929,-4.560000,8.500000,-0.890000
1083930,-4.480000,8.540000,-1.080000


In [65]:
y

,activity
597,Walking
598,Walking
599,Walking
600,Walking
601,Walking
...,...
1083927,Standing
1083928,Standing
1083929,Standing
1083930,Standing


Estandarizar los datos numericos (*StandardScaler)*

*Para el caso de la estandarización de los datos se pueden usar distintas técnicas, como la normalización 01, estandarización entre -1 y 1 o el denominado Z-Score. En este caso usaremos la función Standard Scaler de Python con sus valores por defecto.*


In [76]:
sc = StandardScaler()
X = sc.fit_transform(X)
X

array([[-0.03449447,  0.0769677 ,  0.2664932 ],
       [ 0.01727895,  0.18754729,  0.49575718],
       [-0.29077293,  1.63736749, -0.07583245],
       ...,
       [-1.06166167,  0.16559432, -0.58254798],
       [-1.04645694,  0.17281094, -0.62635849],
       [-1.13388413,  0.1259029 , -0.57101889]])

Como último paso debe reconstruir el dataframe pegando las columnas de los variables predicadores (X) y las categorías (y)


In [78]:
X = pd.DataFrame(X, columns = ["x","y","z"]) # se debe pasar de un arreglo en np a dataframe
X

,x,y,z
0,-0.034494,0.076968,0.266493
1,0.017279,0.187547,0.495757
2,-0.290773,1.637367,-0.075832
3,0.716220,0.573347,-0.650563
4,-0.202758,0.298127,-0.537501
...,...,...,...
290359,-1.143387,0.145749,-0.464951
290360,-1.169995,0.118686,-0.474175
290361,-1.061662,0.165594,-0.582548
290362,-1.046457,0.172811,-0.626358


In [82]:
df_merge = y.merge(X ,left_index=True,right_index=True) #se realiza un merge de los dataframe y , X
df_merge

,activity,x,y,z
597,Walking,-0.086268,0.386590,-0.173191
598,Walking,1.013917,0.782220,-0.474689
599,Walking,0.097528,0.740445,-0.747921
600,Walking,-0.383965,1.416209,-0.210878
601,Walking,-0.391731,-0.898589,-0.509236
...,...,...,...,...
257316,Standing,-0.565607,0.382093,-0.587160
257317,Standing,-0.573210,0.374876,-0.605606
257318,Standing,-0.559906,0.374876,-0.582548
257319,Standing,-0.544701,0.374876,-0.617135
